In [ ]:
import os
import xarray as xr
import numpy as np
import glob


%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime

Get the AM sets

Add the seasonal average of the streams

In [ ]:
def AddSavg(am1, S1_savg, S2_savg, C_savg):
    conditionsALL = [am1.season=='DJF', am1.season=='JJA', am1.season=='MAM', am1.season=='SON']

    choicesS1 = [S1_savg['DJF'], S1_savg['JJA'], S1_savg['MAM'], S1_savg['SON']]
    choicesS2 = [S2_savg['DJF'], S2_savg['JJA'], S2_savg['MAM'], S2_savg['SON']]
    choicesC = [C_savg['DJF'], C_savg['JJA'], C_savg['MAM'], C_savg['SON']]

    am1['S1savg'] = np.select(conditionsALL, choicesS1, default='ERROR').astype(float)
    am1['S2savg'] = np.select(conditionsALL, choicesS2, default='ERROR').astype(float)
    am1['Csavg'] = np.select(conditionsALL, choicesC, default='ERROR').astype(float)
    am1['S1/S1savg'] = am1.Q_S1 / am1.S1savg
    am1['S2/S2savg'] = am1.Q_S2 / am1.S2savg
    am1['C/Csavg'] = am1.Q_C / am1.Csavg
    return am1

In [ ]:
conf = pd.read_csv('Confluences_ok2.csv', delimiter=';')

In [ ]:
#See MetFunctions AM
def AMnew(conf_):
    """
    Creates...
    ds: dictionary with all the Time series per station 
    conf: configuration of the confluences
    c: confluence to be analyzed
    w: window of the multi-day events
    Returns: ds_
    """
    
    #Delete the rows with values lower than 0, or equal to 999
    conf_1 = conf_[~(conf_['Q_S1'] < 0)]
    conf_1 = conf_1[~(conf_1['Q_S2'] < 0)]
    conf_1 = conf_1[~(conf_1['Q_S2'] == 999)]
    conf_1    
    
    #Seasonal averages
    conditions = [
        (conf_1.month==1)|(conf_1.month==2)|(conf_1.month==12),
        (conf_1.month==3)|(conf_1.month==4)|(conf_1.month==5),
        (conf_1.month==6)|(conf_1.month==7)|(conf_1.month==8),
        (conf_1.month==9)|(conf_1.month==10)|(conf_1.month==11)]

    choices = ['DJF', 'MAM', 'JJA', 'SON']
    conf_1['season'] = np.select(conditions, choices, default='ERROR')
    
    savg_S1 = conf_1.groupby(['season'])['Q_S1'].mean()
    savg_S2 = conf_1.groupby(['season'])['Q_S2'].mean()
    savg_C = conf_1.groupby(['season'])['Q_C'].mean()

    
    #2 Extreme selection sets
    AM1_id = conf_1.groupby(['year'])['Q_S1'].idxmax()
    AM2_id = conf_1.groupby(['year'])['Q_S2'].idxmax()
    AM3_id = conf_1.groupby(['year'])['Q_C'].idxmax()

    am1 = conf_1.loc[AM1_id[:]]
    am2 = conf_1.loc[AM2_id[:]]
    am3 = conf_1.loc[AM3_id[:]]
    
    return am1, am2, am3, savg_S1, savg_S2, savg_C

In [ ]:
for c in range(len(conf)):
    print(f'Starting {c} at {str(datetime.now())}')
    op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\01_TSonefile\TS_" + conf.Conf[c] + ".csv" 
    TS = pd.read_csv(op, delimiter=';')
    
    am1, am2, am3, savg_S1, savg_S2, savg_C = AMnew(TS)

    AM1 = AddSavg(am1, savg_S1, savg_S2, savg_C)
    AM2 = AddSavg(am2, savg_S1, savg_S2, savg_C)
    AM3 = AddSavg(am3, savg_S1, savg_S2, savg_C)
    
    sv = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\03_AMhyd" + r"\disc1d_" + conf.Conf[c]
    
    AM1.to_pickle(sv + "_AM1.pkl")
    AM2.to_pickle(sv + "_AM2.pkl")
    AM3.to_pickle(sv + "_AM3.pkl")
    print(f'Finishing {c} at {str(datetime.now().time())}')